In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d kartik2112/fraud-detection

100% 201M/202M [00:01<00:00, 197MB/s]
100% 202M/202M [00:01<00:00, 195MB/s]


In [4]:
import zipfile
zip=zipfile.ZipFile("/content/fraud-detection.zip","r")
zip.extractall('/content')
zip.close()

**Importing required libraries**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [6]:
# loading the dataset to a Pandas DataFrame
train_data = pd.read_csv('/content/fraudTrain.csv')
train_data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [7]:
# loading the dataset to a Pandas DataFrame
test_data = pd.read_csv('/content/fraudTest.csv')
test_data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [8]:
credit_card_data = pd.concat([train_data, test_data], axis = 0)
credit_card_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [9]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852394 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   trans_date_trans_time  object 
 2   cc_num                 int64  
 3   merchant               object 
 4   category               object 
 5   amt                    float64
 6   first                  object 
 7   last                   object 
 8   gender                 object 
 9   street                 object 
 10  city                   object 
 11  state                  object 
 12  zip                    int64  
 13  lat                    float64
 14  long                   float64
 15  city_pop               int64  
 16  job                    object 
 17  dob                    object 
 18  trans_num              object 
 19  unix_time              int64  
 20  merch_lat              float64
 21  merch_long             float64
 22  is_fraud           

In [10]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [11]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['is_fraud'].value_counts()

0    1842743
1       9651
Name: is_fraud, dtype: int64

**This Dataset is highly unblanced**

0 --> Normal Transaction

1 --> fraudulent transaction

In [13]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.is_fraud == 0]
fraud = credit_card_data[credit_card_data.is_fraud == 1]

In [14]:
print(legit.shape)
print(fraud.shape)

(1842743, 23)
(9651, 23)


In [15]:
# statistical measures of the data
legit.amt.describe()

count    1.842743e+06
mean     6.765128e+01
std      1.535481e+02
min      1.000000e+00
25%      9.610000e+00
50%      4.724000e+01
75%      8.256000e+01
max      2.894890e+04
Name: amt, dtype: float64

In [16]:
fraud.amt.describe()

count    9651.000000
mean      530.661412
std       391.028873
min         1.060000
25%       240.075000
50%       390.000000
75%       902.365000
max      1376.040000
Name: amt, dtype: float64

In [17]:
# compare the values for both transactions
credit_card_data.groupby('is_fraud').mean()

<ipython-input-17-a0dda653994e>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  credit_card_data.groupby('is_fraud').mean()


,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,
0,537179.530738,4.174926e+17,67.651278,48817.519363,38.538245,-90.228849,88636.579284,1.358692e+09,38.537950,-90.228935
1,539849.247228,3.970393e+17,530.661412,47999.638379,38.742813,-90.033730,89998.422961,1.355323e+09,38.734962,-90.037919


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 9651

In [18]:
#We will take random 9651 samples from legit data
legit_sample = legit.sample(n=9651)

Concatenating two DataFrames

In [19]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [20]:
new_dataset.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
991286,991286,2020-02-09 04:01:34,4908846471916297,fraud_Spinka-Welch,grocery_net,65.40,Lauren,Torres,F,03030 White Lakes,...,32.2779,-97.2351,5875,"Radiographer, therapeutic",1992-07-24,eb3bdae8c722b961c2d7f30639d4a41a,1360382494,32.792410,-97.221898,0
399496,399496,2019-06-30 21:15:48,4450831335606294,fraud_Schaefer Ltd,kids_pets,125.30,Donna,Davis,F,6760 Donovan Lakes,...,34.5906,-95.3800,1760,Occupational psychologist,1972-01-20,e580fdeec1bdf241336729df70ea73b3,1341090948,34.940002,-95.587325,0
125913,125913,2020-08-04 01:18:25,4661996144291811856,"fraud_Roob, Conn and Tremblay",shopping_pos,62.55,Linda,Park,F,24607 Charles Mountains,...,40.8555,-79.7372,2054,Operations geologist,1963-08-04,ea8689d4e175b6b0c720ee3113dc05f6,1375579105,41.845236,-79.087386,0
441003,441003,2020-12-06 22:14:44,180018375329178,fraud_Schmidt and Sons,shopping_net,30.81,Michelle,Woods,F,952 Joseph Throughway,...,42.3703,-84.2485,2523,Geophysicist/field seismologist,1988-03-21,92f6ce2da48d1ae502f888f4bc1f56d4,1386368084,42.082545,-85.221627,0
796095,796095,2019-12-02 23:34:50,4666314527820883145,fraud_Corwin-Romaguera,travel,475.95,Guy,Thompson,M,5812 Ramos Oval Suite 598,...,41.0233,-98.9041,2202,"Solicitor, Scotland",1974-06-21,6a2f988e9861295a4a466b1b9c19c2d3,1354491290,40.667967,-98.092898,0


In [21]:
new_dataset.tail()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
517197,517197,2020-12-22 22:05:48,2242176657877538,"fraud_Willms, Kris and Bergnaum",shopping_pos,1041.51,Travis,Daniel,M,1327 Rose Causeway Apt. 610,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,35b0297dd026d2e9a75d024a5dec7955,1387749948,34.573471,-89.911011,1
517274,517274,2020-12-22 22:18:07,2242176657877538,fraud_Kuhn LLC,shopping_pos,868.09,Travis,Daniel,M,1327 Rose Causeway Apt. 610,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,da7f67d7375f10a054a3d919448c45dd,1387750687,34.091227,-90.390612,1
517341,517341,2020-12-22 22:31:48,2242176657877538,"fraud_Mosciski, Ziemann and Farrell",shopping_net,1039.42,Travis,Daniel,M,1327 Rose Causeway Apt. 610,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,25b076c7bcd70f272c1c5326bb234f4b,1387751508,34.628434,-90.284780,1
517529,517529,2020-12-22 23:06:03,2242176657877538,fraud_Bauch-Raynor,grocery_pos,289.27,Travis,Daniel,M,1327 Rose Causeway Apt. 610,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,2df7d894868fbc99ec1d8b055585fc9d,1387753563,34.746063,-90.401093,1
517571,517571,2020-12-22 23:13:39,2242176657877538,fraud_Jaskolski-Vandervort,misc_net,766.38,Travis,Daniel,M,1327 Rose Causeway Apt. 610,...,34.6323,-89.8855,14462,Database administrator,1959-03-03,44292cbc51e37dc018ee6a988a4bc426,1387754019,33.771462,-90.651342,1


In [31]:
new_dataset=new_dataset[['Unnamed: 0','amt','is_fraud']]
new_dataset

,Unnamed: 0,amt,is_fraud
991286,991286,65.40,0
399496,399496,125.30,0
125913,125913,62.55,0
441003,441003,30.81,0
796095,796095,475.95,0
...,...,...,...
517197,517197,1041.51,1
517274,517274,868.09,1
517341,517341,1039.42,1
517529,517529,289.27,1


In [32]:
new_dataset['is_fraud'].value_counts()

0    9651
1    9651
Name: is_fraud, dtype: int64

In [33]:
new_dataset.groupby('is_fraud').mean()

,Unnamed: 0,amt
is_fraud,,
0,540263.412496,67.044678
1,539849.247228,530.661412


Splitting the data into Features & Targets

In [34]:
X = new_dataset.drop(columns='is_fraud', axis=1)
Y = new_dataset['is_fraud']

In [35]:
print(X)

        Unnamed: 0      amt
991286      991286    65.40
399496      399496   125.30
125913      125913    62.55
441003      441003    30.81
796095      796095   475.95
...            ...      ...
517197      517197  1041.51
517274      517274   868.09
517341      517341  1039.42
517529      517529   289.27
517571      517571   766.38

[19302 rows x 2 columns]


In [36]:
print(Y)

991286    0
399496    0
125913    0
441003    0
796095    0
         ..
517197    1
517274    1
517341    1
517529    1
517571    1
Name: is_fraud, Length: 19302, dtype: int64


Split the data into Training data & Testing Data

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [38]:
print(X.shape, X_train.shape, X_test.shape)

(19302, 2) (15441, 2) (3861, 2)


**Model Training:**
**Logistic Regression**

In [39]:
model = LogisticRegression()

In [40]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

LogisticRegression()

**Model Evaluation:**
**Accuracy Score**

In [41]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [42]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.7673078168512402


In [43]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [44]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.7681947681947682


In [45]:
print('Training Set f1 score : ', f1_score(Y_train,X_train_prediction))
print('Testing  Set f1 score : ', f1_score(Y_test, X_test_prediction))
print()
print('Test set precision : ', precision_score(Y_test, X_test_prediction))
print('Test set recall    : ', recall_score(Y_test, X_test_prediction))

Training Set f1 score :  0.764316169235815
Testing  Set f1 score :  0.7656454569258969

Test set precision :  0.7739544732662784
Test set recall    :  0.7575129533678756


**Decision Tree Classification**

In [53]:
dt_classifier = DecisionTreeClassifier(max_depth = 50, random_state = 100)
dt_classifier.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=50, random_state=100)

In [54]:
X_train_prediction = dt_classifier.predict(X_train)
X_test_prediction  = dt_classifier.predict(X_test)

print('Training Accuracy : ', accuracy_score(Y_train,X_train_prediction))
print('Testing  Accuracy : ', accuracy_score(Y_test, X_test_prediction))

Training Accuracy :  1.0
Testing  Accuracy :  0.8526288526288526


In [55]:
print('Training Set f1 score : ', f1_score(Y_train,X_train_prediction))
print('Testing  Set f1 score : ', f1_score(Y_test, X_test_prediction))
print()
print('Test set precision : ', precision_score(Y_test, X_test_prediction))
print('Test set recall    : ', recall_score(Y_test, X_test_prediction))

Training Set f1 score :  1.0
Testing  Set f1 score :  0.8520925396412788

Test set precision :  0.854981742305686
Test set recall    :  0.8492227979274611


**Random Forest Classification**

In [56]:
rf_classifier = RandomForestClassifier(n_estimators = 100, max_depth = 12, random_state = 2)
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=12, random_state=2)

In [57]:
X_train_prediction = dt_classifier.predict(X_train)
X_test_prediction  = dt_classifier.predict(X_test)

print('Training Accuracy : ', accuracy_score(Y_train,X_train_prediction))
print('Testing  Accuracy : ', accuracy_score(Y_test, X_test_prediction))

Training Accuracy :  1.0
Testing  Accuracy :  0.8526288526288526


In [58]:
print('Training Set f1 score : ', f1_score(Y_train,X_train_prediction))
print('Testing  Set f1 score : ', f1_score(Y_test, X_test_prediction))
print()
print('Test set precision : ', precision_score(Y_test, X_test_prediction))
print('Test set recall    : ', recall_score(Y_test, X_test_prediction))

Training Set f1 score :  1.0
Testing  Set f1 score :  0.8520925396412788

Test set precision :  0.854981742305686
Test set recall    :  0.8492227979274611


**From above we can conclude that Decision Tree and Random Forest Classification give the optimal accuracy to detect fraudulent credit card transactions.**